# Esercitazione su RNN e LSTM

## Indice contenuti
- [Scopo del progetto](#Scopo-del-progetto)
- [Teoria RNN](#Teoria-RNN)
- [Teoria LSTM](#Teoria-LSTM)
- [Analisi esplorativa del dataset](#Analisi-esplorativa-del-dataset)
    - [Descrizione del dataset](#Descrizione-del-dataset)
    - [Caricamento in memoria del dataset](#Caricamento-in-memoria-del-dataset)
    - [Preprocessing](#Preprocessing)
    - [Definizione della finestra di predizione](#Definizione-della-finestra-di-predizione)
- [Funzioni di attivazione](#Funzioni-di-attivazione)
- [Ottimizzatori](#Ottimizzatori)
- [RNN](#RNN)
    - [Visualizzazione risultati RNN](#Visualizzazione-risultati-RNN)
- [LSTM](#LSTM)
    - [Visualizzazione risultati LSTM](#Visualizzazione-risultati-LSTM)
    - [Visualizzazione RNN vs LSTM](#Visualizzazione-RNN-vs-LSTM)
- [RNN v2](#RNN-v2)
    - [Visualizzazione risultati RNN vs RNN v2 vs Valori reali](#Visualizzazione-risultati-RNN-vs-RNN-v2-vs-Valori-reali)
- [LSTM v2](#LSTM-v2)
    - [Visualizzazione LSTM v2 vs LSTM](#Visualizzazione-LSTM-v2-vs-LSTM)
- [Grafico riassuntivo](#Grafico-riassuntivo)
    
<hr>

## Scopo del progetto

Prevedere le performance del mercato azionario è una delle cose più difficili da fare. Ci sono tanti fattori coinvolti nella previsione: fisici, psicologici, razionali e irrazionali.

Tuttavia, è possibile applicare tecniche di machine learning e deep learning, che combinate con l’utilizzo di indicatori e news sul mercato, rendono più semplice ed
accurata la previsione.

Lo scopo del progetto è la comparazione delle accuracy mostrate da reti neurali quali RNN e LSTM. In particolare, verranno proposti differenti varianti dei modelli citati ottenute modificando gli iperparametri.

La sostanziale differenza esistente tra LSTM e RNN è che la prima riesce a immagazzinare i dati richiedendo più tempo rispetto alla RNN.

## Teoria RNN
Una rete neurale ricorrente (RNN, Recurrent Neural Network), elabora sequenze - derivanti da quotazioni azionarie giornaliere, frasi o misurazioni da sensori - un elemento alla volta, pur conservando un ricordo (chiamato stato) di ciò che è avvenuto in precedenza nella sequenza.

Il termine "ricorrente" significa che l'output nella fase temporale corrente diventa l'input della fase temporale successiva. Ad ogni elemento della sequenza, quindi, il modello considera non solo l'input corrente, ma ciò che ricorda degli stati precedenti.

![image.png](https://miro.medium.com/max/423/1*KljWrINqItHR6ng05ASR8w.png)

Questa memoria consente alla rete di apprendere le dipendenze a lungo termine in una sequenza, il che significa che può prendere in considerazione l'intero contesto quando si effettua una previsione, sia che si tratti della parola successiva in una frase, di una classificazione del sentiment, o della successiva misurazione della temperatura. 

Una RNN è progettata per imitare il modo umano di elaborare le sequenze, pertanto la lettura di un'intera sequenza (rispetto al considerare i singoli elementi di una frase, ad esempio) ci fornisce un contesto per l'elaborazione del suo significato.

## Teoria LSTM
Al centro di un RNN c'è uno strato fatto di celle di memoria. La cella più popolare al momento è la Long Short-Term Memory (LSTM) che mantiene uno stato della cella e un riporto per garantire che il segnale (ovvero le informazioni sotto forma di gradiente) non vadano perse durante l'elaborazione della sequenza. Ad ogni passo temporale, l'LSTM considera la parola corrente, il riporto e lo stato della cella, come mostrato di seguito:

![image.png](https://miro.medium.com/max/1000/1*esTGDR3kcDLaTEHKCBedTQ.png)

L'LSTM ha 3 differenti gate e vettori di peso: c'è un gate “forget” per scartare le informazioni irrilevanti; una porta di "input" per la gestione dell'input corrente e una porta di "output" per produrre previsioni in ogni fase temporale.
La funzione di ogni elemento è in definitiva decisa dai parametri (pesi) che vengono appresi durante la fase di training.

# Analisi esplorativa del dataset

## Descrizione del dataset

Per costruire i differenti modelli di reti neurali, verrà utilizzato il dataset <a href="https://www.kaggle.com/medharawat/google-stock-price">Google Stock Price</a> riguardante dati finanziari del titolo azionario Google.

Di seguito sono riportate le features presenti:
 1. <b>Date</b>: Data di riferimento del titolo
 2. <b>Open</b>: Valore del titolo di apertura
 3. <b>High</b>: Valore più alto del titolo raggiunto in giornata
 4. <b>Low</b>: Valore più basso del titolo raggiunto in giornata
 5. <b>Close</b>: Valore del titolo a chiusura del mercato
 6. <b>Volume</b>: Numero totale di azioni effettivamente negoziate (acquistate e vendute) durante il giorno


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

### Caricamento in memoria del dataset
Con il seguente comando si effettua il caricamento in memoria di quanto contenuto nel dataset _'Train.csv'_.

Per condurre una prima fase di analisi esplorativa e comprendere la natura dei dati a disposizione, si stampano di seguito i primi cinque esempi presenti nel dataset.

Si provvede, successivamente alla considerazione della singola feature _Open_.

Per l'array _train_ si considerano tutti gli esempi, tranne le ultime 50 istanze.
Tali 50 istanze, invece, saranno utilizzate per l'array _test_.

In [ ]:
df = pd.read_csv('../input/google-stock-price/Google_Stock_Price_Train.csv',sep=",")
data = df.loc[:,["Open"]].values

#Del dataset, si considera solo il valore della feature Open. Le restanti feature sono scartate
#Train su tutti gli esempi - gli ultimi 50
train = data[:len(data)-50]
#Test sugli ultimi 50 esempi del dataset
test = data[len(train):]

# reshape
train=train.reshape(train.shape[0],1)

df.head()

## Preprocessing

#### Feature Scaling
Trasforma le features ridimensionando ciascuna feature in un determinato intervallo.
Questo stimatore scala e traduce ogni feature individualmente in modo tale che sia nell'intervallo dato, ad esempio tra zero e uno.

La trasformazione è data da:
$\text{X_std} = (X - X.min(axis=0)) / (X.max(axis=0) - X.min(axis=0))$

$\text{X_scaled} = \text{X_std} * (max - min) + min $

dove min, max = intervallo della feature assunto nel dataset.

Questa trasformazione viene spesso utilizzata come alternativa alla standardizzazione con media zero e varianza unitaria.

In [ ]:
print(f'Intervallo feature Open su train: {train.min()} - {train.max()}')
print(f'Intervallo feature Open su test: {test.min()} - {test.max()}')

from sklearn.preprocessing import MinMaxScaler
#Scalatura dell'intervallo di definizione di Open in [0,1]
scaler = MinMaxScaler(feature_range= (0,1))
train_scaled = scaler.fit_transform(train)


plt.figure(figsize=(15,5))
plt.subplot(1, 2, 1)
plt.title("DataSet prima di MinMaxScaler()")
plt.plot(train)

plt.subplot(1, 2, 2)
plt.title("DataSet dopo MinMaxScaler()")
plt.plot(train_scaled)
plt.show()

## Definizione della finestra di predizione
Per effettuare predizione, è necessario definire una finestra (_window_), che rappresenta il numero di giorni da utilizzare per predire il successivo.

Per comprendere meglio il funzionamento della finestra, si osservi il seguente esempio. Con una finestra di dimensione pari a 50, si predirà il valore del 51-esimo giorno osservando i precedenti 50, e così via.

![](./window_size.jpg)

In [ ]:
# Definizione della finestra di predizione.
X_train = []
y_train = []
window_size = 50

for i in range(window_size, train_scaled.shape[0]):
    X_train.append(train_scaled[i-window_size:i,0])
    y_train.append(train_scaled[i,0])

#Conversione lista -> array
X_train, y_train = np.array(X_train), np.array(y_train)

# Aggiunta della terza dimensione mediante Reshape()
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)  # (1158, 50, 1)

## Funzioni di attivazione

Di seguito vengono riportate le possibili funzioni di attivazione utilizzate per l'aggiornamento dei pesi dei neuroni.


1. <b>relu</b>: La Rectifier Function è la funzione di attivazioni più utilizzata. Restituisce 0 qualora la somma pesata dei segnali in input è minore o uguale a zero, oppure ΣwX negli altri casi. Il codominio della funzione spazia in questo caso da 0 ad infinito.
2. <b>selu</b>: E' una variante della relu. Per valori positivi, restituisce ΣwX, per valori negativi, l'andamento della funzione ricorda il grafico della funzione logaritmo. Può essere utilizzata quando la funzione relu crea il problema noto come "dying relu", ovvero quando tutti gli output assumono tutti lo stesso valore. Questo accade quando si è in presenza di valori molto piccoli, quindi prossimi allo zero. Dato che il gradiente di zero è zero, la rete neurale non è in grado di aggiornare i pesi.
3. <b>tanh</b>: si comporta come la sigmoide, ma il range di valori è [-1, 1]. Il vantaggio è che gli input saranno mappati fortemente in modo negativo, e i valori prossimi allo zero saranno mappati come zero in tanh.
5. <b>sigmoid</b>: in questo caso il codominio della funzione, ovvero i valori che può restituire il neurone, spazia tra 0 ed 1 in un intervallo continuo. Infatti, la caratteristica di questa funzione è che smussata. Può essere utilizzata al posto della Threshold Function considerando il valore in uscita non come Y ma come probabilità che Y sia uguale ad uno, ovvero Prob(Y=1).

## Ottimizzatori

1. <b>RMSprop</b>: è raccomandabile lasciare i parametri di default di questa funzione a eccezione del learning rate. Sono stati fatti dei test con differenti valori di learning rate e selezionato il migliore per il confronto (lr = 0.001)
2. <b>Adam</b>: è una combinazione di RMSprop e lo Stochastic Gradient Descent con momentum. Questo usa una media mobile quadratica esponenziale vt dei precedenti gradienti per scalare il valore di learning rate e una media mobile esponenziale mt dei precedenti gradienti per una stima del momentum.
3. <b>AdaMax</b>: è una variante di Adam che applica la norma infinita l∞ alla media mobile quadratica esponenziale vt per scalare il learning rate
4. <b>Nadam</b>: è una combinazione di RMSprop e momentum. Nadam utilizza il valore vt di Adam per il learning rate, mentre modifica il valore del momentum mt applicando il gradiente accelerato di Nesterov. Per valori di learning rate piccoli, Nesterov diventa equivalente al normale momentum e, quindi, non ci si aspettano grandi cambiamenti di performance rispetto ad Adam.

Nella creazione dei modelli, viene utilizzato il Dropout, una tecnica di regolarizzazione per reti neurali. Dropout seleziona casualmente un numero di neuroni da ignorare durante il training. Questo significa che il contributo di questi neuroni per l’attivazione degli strati inferiori viene temporaneamente rimosso. L’effetto prodotto è quello di una rete neurale meno sensibile a specifici pesi dei neuroni, ottenendo un migliore generalizzazione che previene sovradattamento dei dati di training.
Generalmente si sceglie un dropout compreso tra il 20 e il 50% e, ovviamente, sarà necessario usare una rete più grande poiché alcuni neuroni non verranno usati.

<a id="4"></a>
## RNN
Una rete neurale ricorrente è semplicemente un tipo di rete neurale densamente connessa. Tuttavia, la differenza fondamentale rispetto alle normali reti feed forward è l'introduzione del tempo: in particolare, l'output dello strato nascosto in una rete neurale ricorrente viene reinserito in se stesso, come di seguito rappresentato, permettendo di trattare dati dipendenti dal tempo:

![](https://i0.wp.com/adventuresinmachinelearning.com/wp-content/uploads/2017/09/Explicit-RNN.jpg?w=363&ssl=1)

L'approccio standard per l'utilizzo di reti neurali ricorrenti è poco utilizzato nella pratica. Il motivo principale è legato al calcolo del gradiente.

Per le reti neurali ricorrenti, idealmente, vorremmo avere una lunga memoria, in modo che la rete possa connettere relazioni di dati a distanze significative nel tempo. Questo tipo di rete potrebbe fare reali progressi nella comprensione di come sono correlati gli eventi del mercato azionario, ad esempio. Tuttavia, più passi di tempo si hanno, più possibilità si ha che i gradienti di retro propagazione si accumulino ed esplodano o svaniscano nel nulla.

Potremmo usare le funzioni di attivazione ReLU per ridurre questo problema, ma non eliminarlo definitivamente. Tuttavia, il modo più diffuso per affrontare questo problema nelle reti neurali ricorrenti è l'utilizzo di reti LSTM (Long-Short Term Memory).

Nel seguente snippet è riportato l'utilizzo di RNN utilizzando il metodo _Sequential()_ messo a disposizione dalla libreria Keras.

In particolare, la struttura della rete neurale è così composta:
- SimpleRNN: con 50 unità e funzione di attivazione _tanh_. Il valore di dropout (per evitare overfitting) è posto a 0.2
- SimpleRNN: con 50 unità e funzione di attivazione _tanh_. Il valore di dropout (per evitare overfitting) è posto a 0.2
- SimpleRNN: con 50 unità e funzione di attivazione _tanh_. Il valore di dropout (per evitare overfitting) è posto a 0.2
- SimpleRNN: con 50 unità e funzione di attivazione _tanh_. Il valore di dropout (per evitare overfitting) è posto a 0.2
- Dense: layer di output, formato da una sola unità

Al fine di compilare il modello creato, si utilizza come funzione di loss _mean_squared_error_ mentre, come ottimizzatore, si sceglie uno tra quelli riportati precedentemente.

In [ ]:
from keras.models import Sequential  
from keras.layers import Dense 
from keras.layers import SimpleRNN
from keras.layers import Dropout

# Definizione del modello di RNN
model = Sequential()

#Definizione del primo layer con l'aggiunta della regolarizzazione mediante Dropout
model.add(SimpleRNN(units = 50,activation='tanh', return_sequences = True, input_shape = (X_train.shape[1], 1)))
model.add(Dropout(0.2)) #Spegne il 20% dei neuroni

#Definizione del secondo layer con l'aggiunta della regolarizzazione mediante Dropout
model.add(SimpleRNN(units = 50,activation='tanh', return_sequences = True))
model.add(Dropout(0.2))

#Definizione del terzo layer con l'aggiunta della regolarizzazione mediante Dropout
model.add(SimpleRNN(units = 50,activation='tanh', return_sequences = True))
model.add(Dropout(0.2))

#Definizione del quarto layer con l'aggiunta della regolarizzazione mediante Dropout
model.add(SimpleRNN(units = 50))
model.add(Dropout(0.2))

# Definizione Output Layer
model.add(Dense(units = 1))

# Compilazione della RNN
model.compile(optimizer = 'adam', loss = 'mean_squared_error')

# Fit di RNN
#batch_size: numero di esempi di training da prendere in considerazione
model.fit(X_train, y_train, epochs = 100, batch_size = 32)

In [ ]:
inputs = data[len(data) - len(test) - window_size:]
#Utilizzo di Min Max Scaler per scalare i dati presenti in inputs
inputs = scaler.transform(inputs)

In [ ]:
X_test = []
for i in range(window_size, inputs.shape[0]):
    #Si assumono 50 esempi da 0 a 50, da 1 a 51.
    X_test.append(inputs[i-window_size:i, 0]) 
X_test = np.array(X_test)
#Trasformazione in dimensioni compatibili con il tensore
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

In [ ]:
predicted_data = model.predict(X_test)
#Inverse_Transoform: Annulla lo scaling di predicted_data in base al range di definizione della feature
predicted_data = scaler.inverse_transform(predicted_data)

### Visualizzazione risultati RNN


In [ ]:
plt.figure(figsize=(8,4), dpi=80, facecolor='w', edgecolor='k')
plt.plot(test,color="orange",label="Real value")
plt.plot(predicted_data,color="c",label="RNN predicted result")
plt.legend()
plt.xlabel("Giorni")
plt.ylabel("Valore di apertura")
plt.grid(True)
plt.show()

## LSTM
Per ridurre il problema del calcolo del gradiente e quindi consentire alle reti più profonde e alle reti neurali ricorrenti di funzionare bene in contesti pratici, è necessario un modo per ridurre la moltiplicazione dei gradienti che sono inferiori a zero. 

La cella LSTM è un'unità logica appositamente progettata che aiuterà a ridurre il problema del calcolo del gradiente in modo sufficiente da rendere le reti neurali ricorrenti più utili per le attività di memoria a lungo termine. 

Il modo in cui lo fa è creando uno stato di memoria interna che viene semplicemente aggiunto all'input processato, il che riduce notevolmente l'effetto moltiplicativo di gradienti piccoli (tendenti a 0). La dipendenza dal tempo e gli effetti degli input precedenti sono controllati da un concetto interessante chiamato dimenticare "forget rate", che determina quali stati vengono ricordati o dimenticati. Sono definite altre dye porte: la porta di ingresso e la porta di uscita.

![](https://i1.wp.com/adventuresinmachinelearning.com/wp-content/uploads/2017/09/LSTM-diagram.png?w=669&ssl=1)

Il flusso di dati è da sinistra a destra nel diagramma riportato, con l'input corrente $x_t$ e l'output della cella precedente $h_{t-1}$ concatenati insieme e che entrano nella "sezione dei dati" superiore.

Nel seguente snippet è riportato l'utilizzo di LSTM utilizzando il metodo _Sequential()_ messo a disposizione dalla libreria Keras.

In particolare, la struttura della rete neurale LSTM è così composta:
- LSTM: con 10 nodi per l'unità
- Dense: layer di output, formato da una sola unità

Al fine di compilare il modello creato, si utilizza come funzione di loss _mean_squared_error_ mentre, come ottimizzatore, si sceglie uno tra i seguenti riportati:
1. <b>RMSprop</b>: è raccomandabile lasciare i parametri di default di questa funzione a eccezione del learning rate. Sono stati fatti dei test con differenti valori di learning rate e selezionato il migliore per il confronto (lr = 0.001)
2. <b>Adam</b>: è una combinazione di RMSprop e lo Stochastic Gradient Descent con momentum. Questo usa una media mobile quadratica esponenziale vt dei precedenti gradienti per scalare il valore di learning rate e una media mobile esponenziale mt dei precedenti gradienti per una stima del momentum.
3. <b>AdaMax</b>: è una variante di Adam che applica la norma infinita l∞ alla media mobile quadratica esponenziale vt per scalare il learning rate
4. <b>Nadam</b>: è una combinazione di RMSprop e momentum. Nadam utilizza il valore vt di Adam per il learning rate, mentre modifica il valore del momentum mt applicando il gradiente accelerato di Nesterov. Per valori di learning rate piccoli, Nesterov diventa equivalente al normale momentum e, quindi, non ci si aspettano grandi cambiamenti di performance rispetto ad Adam.

Il modello è fittato con l'utilizzo di 50 epoche ed una _batch_size_ impostata ad 1.

In [ ]:
# Import delle librerie
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

In [ ]:
model = Sequential()
model.add(LSTM(10, input_shape=(None,1)))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(X_train, y_train, epochs=50, batch_size=1)

In [ ]:
predicted_data2=model.predict(X_test)
predicted_data2=scaler.inverse_transform(predicted_data2)

### Visualizzazione risultati LSTM

In [ ]:
plt.figure(figsize=(8,4), dpi=80, facecolor='w', edgecolor='k')
plt.plot(test,color="LimeGreen",label="Real values")
plt.plot(predicted_data2,color="Gold",label="Predicted LSTM result")
plt.legend()
plt.xlabel("Giorni")
plt.ylabel("Valore di apertura")
plt.grid(True)
plt.show()

### Visualizzazione RNN vs LSTM

In [ ]:
plt.figure(figsize=(8,4), dpi=80, facecolor='w', edgecolor='k')
plt.plot(test,color="green", linestyle='dashed',label="Valori reali")
plt.plot(predicted_data2,color="blue", label="LSTM predicted result")
plt.plot(predicted_data,color="red",label="RNN predicted result")
plt.legend()
plt.xlabel("Giorni")
plt.ylabel("Valore di apertura")
plt.grid(True)
plt.show()

Nel precedente grafico, è stata comparata la curva predetta di LSTM e posta a confronto con quanto ottenuto con RNN, rispetto ai valori reali presenti nel dataset.

In particolare, osservando il grafico di LSTM risulta più vicino ai valori reali, rispetto a quanto ottenuto mediante RNN.

Di seguito, invece, si effettueranno differenti tentativi di cambio di iperparametri per entrambi i modelli (come ad esempio il numero di unità, di layer, epoche, batch_size e funzione di attivazione), al fine di ottimizzare il risultato ottenuto sia per LSTM sia per RNN.

### RNN v2
Rispetto alla precedente RNN, si effettuano i seguenti cambiamenti:
- Primo layer: il numero di unità passa da 50 a 100. La funzione di attivazione passa da 'tanh' a 'relu'
- Secondo layer: il numero di unità rimane inviariato
- Cancellazione del secondo e terzo layer intermedi
- Il modello è fittato con un numero di epoche pari a 500 (rispetto alle precedenti 100)
- La dimensione del batch_size è diminuita a 16 (rispetto alla precedente dimensione 32)

In [ ]:
#Import delle librerie
from keras.models import Sequential  
from keras.layers import Dense 
from keras.layers import SimpleRNN
from keras.layers import Dropout 

# Definizione del modello di RNN
model = Sequential()

#Definizione del primo layer con l'aggiunta della regolarizzazione mediante Dropout
model.add(SimpleRNN(units = 100,activation='relu', return_sequences = True, input_shape = (X_train.shape[1], 1)))
model.add(Dropout(0.2))

#Definizione del secondo layer con l'aggiunta della regolarizzazione mediante Dropout
model.add(SimpleRNN(units = 50))
model.add(Dropout(0.2))


# Definizione Output Layer
model.add(Dense(units = 1)) 

# Compilazione della RNN
model.compile(optimizer = 'adam', loss = 'mean_squared_error')

# Fit di RNN
#batch_size: numero di esempi di training da prendere in considerazione
model.fit(X_train, y_train, epochs = 500, batch_size = 16)

In [ ]:
predicted_data_modified = model.predict(X_test)
predicted_data_modified = scaler.inverse_transform(predicted_data_modified)

<a id="10"></a>
### Visualizzazione risultati RNN vs RNN v2 vs Valori reali

In [ ]:
plt.figure(figsize=(8,4), dpi=80, facecolor='w', edgecolor='k')
plt.plot(test,color="gray",label="Real values")
plt.plot(predicted_data,color="cyan",label="RNN result v1")
plt.plot(predicted_data_modified,color="blue",label="RNN result v2")

plt.legend()
plt.xlabel("Giorni")
plt.ylabel("Valore di apertura")
plt.grid(True)
plt.show()

Nel precedente grafico, è stata comparata la curva predetta di RNN (il primo modello creato) rispetto alla seconda versione di RNN e posta a confronto con quanto ottenuto con i valori reali presenti nel dataset.

In particolare, osservando il grafico di RNN v2, si ha un miglioramento delle predizioni rispetto alla prima versione e risulta più vicino ai valori reali presenti nel dataset.

Di seguito, invece, si apportano modifiche anche all'implementazione di LSTM.

### LSTM v2
Rispetto alla precedente LSTM, si effettuano i seguenti cambiamenti:
- Il modello è fittato con un numero di epoche pari a 200 (rispetto alle precedenti 50)
- La dimensione del batch_size è aumentata a 4 (rispetto alla precedente dimensione 1)

In [ ]:
#Import delle librerie
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler 


model = Sequential()
model.add(LSTM(10, input_shape=(None,1)))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')
model.fit(X_train, y_train, epochs=200, batch_size=4)

In [ ]:
predicted_data2_modified=model.predict(X_test)
predicted_data2_modified=scaler.inverse_transform(predicted_data2_modified)

<a id="12"></a>
### Visualizzazione LSTM v2 vs LSTM

In [ ]:
plt.figure(figsize=(8,4), dpi=80, facecolor='w', edgecolor='k')
plt.plot(test,color="DimGray",label="Real values", linestyle="dashed")
plt.plot(predicted_data2,color="Magenta",label="LSTM predicted")
plt.plot(predicted_data2_modified,color="c", label="LSTM v2 predicted")
plt.legend()
plt.xlabel("Giorni")
plt.ylabel("Valore di apertura")
plt.grid(True)
plt.show()

Nel precedente grafico, è stata comparata la curva predetta della seconda versione di LSTM e posta a confronto con quanto ottenuto con LSTM (il primo modello creato), rispetto ai valori reali presenti nel dataset.

In particolare, osservando il grafico di LSTM v2 risulta molto simile a quanto ottenuto per LSTM. Si può bnotare solo qualche discostamento nel grafico che porta a scegliere LSTM v2.

### Grafico riassuntivo

In [ ]:
plt.figure(figsize=(16,8), dpi=80, facecolor='w', edgecolor='k')
plt.plot(test,color="DimGray",label="Real value", linestyle="dashed")
plt.plot(predicted_data2,color="blue",label="LSTM predicted")
plt.plot(predicted_data2_modified,color="red", linestyle="dashed", label="LSTM Modified predicted")
plt.plot(predicted_data,color="c",label="RNN predicted")
plt.plot(predicted_data_modified,color="green", linestyle="dashed", label="RNN modified predicted")
plt.legend()
plt.xlabel("Giorni")
plt.ylabel("Valore di apertura")
plt.grid(True)
plt.show()

Nel precedente grafico, sono state riportate le quattro differenti curve predette dei rispettivi quattro modelli che sono stati ideati e perfezionati.

Dall'analisi delle curve, è possibile osservare che vi è un modello che segue poco fedelmente i valori reali (RNN v1). 
L'implementazione di RNN v2, invece, segue l'andamento dei valori reali anche se con un netto scostamento temporale.

Le predizioni migliorano con le versioni "alterate" che hanno portato entrambe ad un incremento dell'accuracy registrata rispetto ai valori reali. In particolare, è possibile affermare che LSTM v2 risulta migliore per la predizione dei valori finanziari rispetto al modello RNN.